In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Visual_tools as VTools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list)) # insert your file
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1640.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1440.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0640.raw', 'C://Users//jony//Programming//Python//

In [2]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(VTools)
import matplotlib.pyplot as plt
interval = 100 # Set your monitoring chunck size
file = file_path[-1] # Select the file
fReader = rawFileReader.fileReader(file) # load in the classs object
fReader.skip_events(900*interval)
print(file)
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)
file_name = file.split("//")[-1][:-4] #must be a right file
    

import cv2
import os

image_folder = 'images_video'
video_name = f'{file_name}_bad.mp4'



with tqdm(total=max_process_event_chunk*interval, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        try:
            event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        except:
            print(f'Error at event chunk {processedEvents}')
            continue
        #Zone of Reconstruction
        if event_chunk:
            for evt_num, evt in enumerate(event_chunk):
                #[print(hit) for hit in VTools.all_hits_event(evt)]
                maps = VTools.hitHeatMap(evt)
                for rpc in range(6):
                    plt.imshow(maps[rpc], interpolation='nearest')
                    plt.title(f'Event {evt_num} RPC {rpc}')
                    plt.savefig(f'images_video/{processedEvents*100+10*evt_num+rpc}.png')
                pbar.update(1)

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape


video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

Skipping Events: 100%|██████████| 90000/90000 [00:10<00:00, 8412.45Events/s] 


C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw


Processing Events: 100%|██████████| 100/100 [36:10<00:00, 21.71s/Events]
